In [25]:
import numpy as np
import pandas as pd

from io import StringIO
import requests

In [26]:
#This file will scrape the government of website for climate data for the specified years.

In [27]:
#Data station data somewhat incomplete. Use data from three stations instead.
SI_A = 833 #N Vancouver Wharves station 5.83 km from the cafe
SI_B = 853 #Richmond Dallyn 2 station 11.87 km from the cafe
SI_C = 837 #Richmond Nature Park station 12.40 km from the cafe

#http://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=833&Year=2018&Month=1&Day=1&timeframe=2&submit=Download+Data

#Gets daily climate data at specified station. Month defaulted to 1 because each year csv contains data for all months


# Skip rows sometimes 24 and 25. Since blank line after 24, 24 works for 2017 and 2018.
# However - for the sake of learning, we're gonna make it dynamic.


def getSkipRow(stationID,year):
    base = "http://climate.weather.gc.ca/climate_data/bulk_data_e.html?"
    query = "format=csv&stationID={}&Year={}&Month=1&Day=1&timeframe=2".format(stationID, year)
    endpoint = base + query
    header = 'Date/Time'
    
    full_text = requests.get(endpoint).text
    
    row_num = 0
    for i,text in enumerate(full_text.splitlines()):
        if header in text:
            row_num = i
            break
    
    return StringIO(full_text),row_num
    

def getDailyClimateData(stationID, year):
    temp_file, row_number = getSkipRow(stationID, year)   
    return pd.read_csv(temp_file, skiprows = row_number)



In [28]:
#Three different dataframe for stations A, B, and C.
from_year = 2017
end_year = 2018+1
year_range = np.arange(from_year, end_year, 1)

#Valuable lesson after debugging: set ignore_index to be true if index meaningless or else it counts from 0 again
dfA = pd.concat((getDailyClimateData(stationID = SI_A, year=x) for x in year_range), ignore_index = True)
dfB = pd.concat((getDailyClimateData(stationID = SI_B, year=x) for x in year_range), ignore_index = True)
dfC = pd.concat((getDailyClimateData(stationID = SI_C, year=x) for x in year_range), ignore_index = True)


In [29]:

#Fill empty rain data in df with df2's, then with df3's if still missing.
dfA.loc[dfA['Total Rain (mm)'].isnull(),['Total Rain (mm)']]=dfB.iloc[dfA.loc[dfA['Total Rain (mm)'].isnull()].index]['Total Rain (mm)']
dfA.loc[dfA['Total Rain (mm)'].isnull(),['Total Rain (mm)']]=dfC.iloc[dfA.loc[dfA['Total Rain (mm)'].isnull()].index]['Total Rain (mm)']

dfA.loc[dfA['Mean Temp (°C)'].isnull(),['Mean Temp (°C)']]=dfC.iloc[dfA.loc[dfA['Mean Temp (°C)'].isnull()].index]['Mean Temp (°C)']

dfA.loc[dfA['Total Snow (cm)'].isnull(),['Total Snow (cm)']]=dfB.iloc[dfA.loc[dfA['Total Snow (cm)'].isnull()].index]['Total Snow (cm)']
dfA.loc[dfA['Total Snow (cm)'].isnull(),['Total Snow (cm)']]=dfC.iloc[dfA.loc[dfA['Total Snow (cm)'].isnull()].index]['Total Snow (cm)']


In [30]:
#Now count the number of rows containing null
dfA=dfA[['Date/Time','Mean Temp (°C)','Total Rain (mm)','Total Snow (cm)']]
dfA.isnull().any(axis=1).sum()


21

In [31]:
#21 for 2017-2018, which we can live with.

In [32]:
dfA['Date/Time'] = pd.to_datetime(dfA['Date/Time'])
dfA = dfA.rename(index=str, columns={'Date/Time':'Date'})
dfA = dfA.set_index('Date')
df_weather = dfA

In [33]:
df_weather

,Mean Temp (°C),Total Rain (mm),Total Snow (cm)
Date,,,
2017-01-01,0.3,0.0,0.0
2017-01-02,-1.8,0.0,0.0
2017-01-03,-5.5,0.0,0.0
2017-01-04,-4.0,0.0,0.0
2017-01-05,-3.8,0.0,0.0
2017-01-06,-0.5,1.0,0.0
2017-01-07,-2.5,0.0,0.0
2017-01-08,1.3,18.0,0.0
2017-01-09,2.5,2.0,0.0


In [68]:
# all(df_weather['Mean Temp (°C)'].astype(str).apply(lambda x: all(ord(y)<128 for y in x)))
# all(df_weather['Total Rain (mm)'].astype(str).apply(lambda x: all(ord(y)<128 for y in x)))
# all(df_weather['Total Snow (cm)'].astype(str).apply(lambda x: all(ord(y)<128 for y in x)))
#Just checking if each column contains all non-ascii characters. They do, but read csv doesnt work...
#Saving with encoding specified for this reason.

In [67]:
#We'll save this just because we don't want to import Jupyter notebook.
df_weather.to_csv('Clean Weather (2017-2018).csv', encoding='utf8')